# Carregando documentos e ampliando o contexto para resposta


In [1]:
# Carregando variáveis de ambiente
from dotenv import load_dotenv, find_dotenv
import os
load_dotenv(find_dotenv(), override=True)

True

In [3]:
# Inicializando a sessão com o Astra/Cassandra
# A criação da sessão é realizada pelos arquivos cqlsession.py e connect_database.py
from cqlsession import getCQLSession, getCQLKeyspace
cqlMode = 'astra_db' # 'astra_db'/'local'
session = getCQLSession(mode=cqlMode)
keyspace = getCQLKeyspace(mode=cqlMode)

In [4]:
# Neste exemplo, vou utilizar os embeddings da OpenAI.
# Aqui, importamos o LLM e os Embeddings
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings
llm = OpenAI(temperature=0)
embedding_func = OpenAIEmbeddings()

In [5]:
# Importando o vector store Cassandra (vamos usar o Astra, mas é o Cassandra)
from langchain.vectorstores.cassandra import Cassandra

In [8]:
# Importando o índice, separadores de texto, e auxiliares para processamendo dos documentos
from langchain.indexes import VectorstoreIndexCreator
from langchain.text_splitter import (
    CharacterTextSplitter,
    RecursiveCharacterTextSplitter,
)
from langchain.docstore.document import Document
from langchain.document_loaders import TextLoader

In [11]:
loader = TextLoader('./o_alienista.txt')
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=40)
docs = text_splitter.split_documents(documents)

In [13]:
# Criando a tabela para armezenar os vetores

table_name = 'astra_vector_ex3_openai'
keyspace_name = 'demo'

docsearch = Cassandra.from_documents(
    documents=docs,
    embedding=embedding_func,
    session=session,
    keyspace=keyspace_name,
    table_name=table_name,
)

In [21]:
# Definindo a busca e  procurando os itens similares
query = "Quem foi Simão Bacamarti?"
docs = docsearch.similarity_search(query)

In [23]:
# Consultando os elementos retornados pela query de similaridade
for i, d in enumerate(docs):
    print(f"\n## Document {i}\n")
    print(d.page_content)



## Document 0

Mas nada escapa a um espírito original e penetrante. Simão Bacamarte, que desde algum tempo notava o zelo, a sagacidade, a paciência, a moderação daquele agente, reconheceu a habilidade e o tino com que ele levara a cabo uma experiência tão melindrosa e complicada, e determinou recolhê-lo imediatamente à Casa Verde; deu-lhe todavia um dos melhores cubículos.

## Document 1

Era decisivo. Simão Bacamarte curvou a cabeça juntamente alegre e triste, e ainda mais alegre do que triste. Ato continuo, recolheu-se à Casa Verde. Em vão a mulher e os amigos lhe disseram que ficasse, que estava perfeitamente são e equilibrado: nem rogos nem sugestões nem lágrimas o detiveram um só instante.

## Document 2

Não eram gritos na rua, eram suspiros em casa, mas não tardava a hora dos gritos. O terror crescia; avizinhava-se a rebelião. A idéia de uma petição ao governo, para que Simão Bacamarte fosse capturado e deportado, andou por algumas cabeças, antes que o barbeiro Porfírio a expen

In [24]:
# Alem da busca por similaridade, podemos ainda buscar mais conteúdo similar e, entre este, escolhar os mais diferentes, 
# aumentando a diversidade e riqueza do conteúdo.
# Isto se faz utilizando o MMR (Maximal Marginal Relevance)

retriever = docsearch.as_retriever(search_type="mmr")
matched_docs = retriever.get_relevant_documents(query)
for i, d in enumerate(matched_docs):
    print(f"\n## Document {i}\n")
    print(d.page_content)


## Document 0

Mas nada escapa a um espírito original e penetrante. Simão Bacamarte, que desde algum tempo notava o zelo, a sagacidade, a paciência, a moderação daquele agente, reconheceu a habilidade e o tino com que ele levara a cabo uma experiência tão melindrosa e complicada, e determinou recolhê-lo imediatamente à Casa Verde; deu-lhe todavia um dos melhores cubículos.

## Document 1

As crônicas da vila de Itaguaí dizem que em tempos remotos vivera ali um certo médico, o Dr. Simão Bacamarte, filho da nobreza da terra e o maior dos médicos do Brasil, de Portugal e das Espanhas. Estudara em Coimbra e Pádua. Aos trinta e quatro anos regressou ao Brasil, não podendo el-rei alcançar dele que ficasse em Coimbra, regendo a universidade, ou em Lisboa, expedindo os negócios da

## Document 2

Não eram gritos na rua, eram suspiros em casa, mas não tardava a hora dos gritos. O terror crescia; avizinhava-se a rebelião. A idéia de uma petição ao governo, para que Simão Bacamarte fosse captura

In [27]:
# De maneira mais simples, podemos fazer a mesma busca assim:
found_docs = docsearch.max_marginal_relevance_search(query, k=4, fetch_k=15)
for i, doc in enumerate(found_docs):
    print(f"{i + 1}.", doc.page_content, "\n")

Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600).


1. Mas nada escapa a um espírito original e penetrante. Simão Bacamarte, que desde algum tempo notava o zelo, a sagacidade, a paciência, a moderação daquele agente, reconheceu a habilidade e o tino com que ele levara a cabo uma experiência tão melindrosa e complicada, e determinou recolhê-lo imediatamente à Casa Verde; deu-lhe todavia um dos melhores cubículos. 

2. As crônicas da vila de Itaguaí dizem que em tempos remotos vivera ali um certo médico, o Dr. Simão Bacamarte, filho da nobreza da terra e o maior dos médicos do Brasil, de Portugal e das Espanhas. Estudara em Coimbra e Pádua. Aos trinta e quatro anos regressou ao Brasil, não podendo el-rei alcançar dele que ficasse em Coimbra, regendo a universidade, ou em Lisboa, expedindo os negócios da 

3. Não eram gritos na rua, eram suspiros em casa, mas não tardava a hora dos gritos. O terror crescia; avizinhava-se a rebelião. A idéia de uma petição ao governo, para que Simão Bacamarte fosse capturado e deportado, andou por algumas c

In [29]:
# Agora, vamos usar este contexto para enriquecer as respostas

from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

llmChat = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.5)
retriever = docsearch.as_retriever(search_type="mmr", search_kwargs={'k':3,'fetch_k':10})
chain = RetrievalQA.from_chain_type(llm=llmChat, chain_type='stuff', retriever=retriever)

In [37]:
chat_query = 'Quem foi simão bacamarti?'
answer = chain.run(query=chat_query)
print(answer)

A Casa Verde era um asilo, cujo nome foi dado devido à cor verde das janelas, que era algo inédito em Itaguaí. Foi inaugurada com grande pompa e atraiu pessoas de várias vilas e cidades próximas, inclusive do Rio de Janeiro. Muitos pacientes já estavam alojados no local. No entanto, um médico sem clínica afirmou que a Casa Verde era um cárcere privado. D. Evarista ficou chocada ao descobrir que Martim Brito, um jovem com ideias bonitas, havia sido alojado lá. As duas senhoras atribuíram essa ação aos ciúmes do alienista.


# Adicionando memória